# Efficient Yelp API Calls by Wesley Giles

## Import necessary modules and environment variables

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
from dotenv import load_dotenv
load_dotenv()

True

## Now to create the Yelp Client

In [3]:
yelp = YelpAPI(os.environ["YELP_API_KEY"],timeout_s=5.0)
yelp

## Make the file path for the json file

In [11]:
def add_results(file_name,location,term):
  try:
    os.makedirs("./data/")
  except:
    pass
  path = f"./data/{file_name}"
  try:
    with open(path,"r") as f:
      results = json.load(f)
  except:
    print("Initializing new file as empty list")
    with open(path,"w") as f:
      f.writelines(["[]"])
      results = []
  new_results = yelp.search_query(location=location,
                                  term=term,
                                  offset=len(results))
  page_amount = len(new_results["businesses"])
  num_pages = math.ceil((new_results["total"] - len(results)) / page_amount)
  for i in tqdm_notebook(range(num_pages)):
    time.sleep(0.2)
    with open(path,"w") as f:
      results.extend(new_results["businesses"])
      json.dump(results,f)
      new_results = yelp.search_query(location=location,
                                  term=term,
                                  offset=len(results))
  return results

In [12]:
add_results("results.json","43017","Restauraunt")

Initializing new file as empty list


  0%|          | 0/1 [00:00<?, ?it/s]